In [1]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
from sentence_transformers import SentenceTransformer
from cuBERTopic import gpu_bertopic
import cudf

docs = fetch_20newsgroups(subset='all')['data']

In [2]:
model_sbert = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model_sbert.encode(
    docs,
    show_progress_bar=True,
    batch_size=64,
    convert_to_numpy=True,
)

Batches:   0%|          | 0/295 [00:00<?, ?it/s]

In [3]:
topic_model = BERTopic()
topics_cpu, probs_cpu = topic_model.fit_transform(docs, embeddings)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [4]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,6104,-1_file_program_jpeg_available
1,0,300,0_gun_guns_firearms_militia
2,1,235,1_clayton_cramer_gay_homosexual
3,2,233,2_fire_fbi_atf_compound
4,3,196,3_israeli_israel_arab_arabs
...,...,...,...
362,361,10,361_osiris_masonry_mithras_egypt
363,362,10,362_cages_doubleparking_ggrrrrrr_34211castleed...
364,363,10,363_biggest_disappointment_mvp_surprise
365,364,10,364_cable_disk_internal_scsi


In [5]:
topic_model.get_topic(0)

[('gun', 0.015695390675989157),
 ('guns', 0.009421003621008022),
 ('firearms', 0.007910979846550816),
 ('militia', 0.0066126289883310435),
 ('amendment', 0.006244535644699449),
 ('crime', 0.005471696868122501),
 ('handgun', 0.005365190429149443),
 ('control', 0.005311050033919601),
 ('firearm', 0.005001228451430933),
 ('arms', 0.0048916521514328)]

In [6]:
gpu_topic = gpu_bertopic()
topics_gpu, probs_gpu = gpu_topic.fit_transform(docs)

Label prop iterations: 22
Label prop iterations: 10
Label prop iterations: 5
Label prop iterations: 5
Label prop iterations: 3
Label prop iterations: 2
Iterations: 6
4809,175,468,15,364,1607
Label prop iterations: 4
Iterations: 1
3385,50,130,4,67,166
Label prop iterations: 2
Iterations: 1
3935,51,131,4,64,111


In [7]:
gpu_topic.get_topic_info()

,Topic,Count,Name
136,-1,6540,-1_file_email_program_thanks
205,86,801,86_baseball_game_team_year
156,124,670,124_clipper_encryption_key_chip
41,187,357,187_gun_guns_militia_firearms
302,166,332,166_fbi_fire_batf_compound
...,...,...,...
313,31,10,31_audio_relays_switching_clicks
315,206,10,206_print_printer_file_drag
316,236,10,236_biggest_disappointment_mvp_surprise
320,208,10,208_com4_com_com3_ports


In [8]:
gpu_topic.get_topic(187)

[('gun', array(0.01414468)),
 ('guns', array(0.00932788)),
 ('militia', array(0.00753051)),
 ('firearms', array(0.00646954)),
 ('weapons', array(0.00624489)),
 ('amendment', array(0.00568526)),
 ('weapon', array(0.00506543)),
 ('crime', array(0.00500248)),
 ('arms', array(0.00498124)),
 ('control', array(0.00426253))]